<a href="https://colab.research.google.com/github/MissoumYoucef/Kaggle_Competition/blob/main/Facial_Keypoints_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.layers import Conv2D,Dropout,Dense,Flatten
from keras.models import Sequential
from keras.layers import LeakyReLU
from keras.models import Sequential, Model
from keras.layers import Activation, Convolution2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout, Conv2D,MaxPool2D, ZeroPadding2D
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
train_data = pd.read_csv('training.csv')
test_data = pd.read_csv('test.csv')

In [ ]:
train_data.shape,test_data.shape

((7049, 31), (1783, 2))

In [ ]:
train_data.columns

Index(['left_eye_center_x', 'left_eye_center_y', 'right_eye_center_x',
       'right_eye_center_y', 'left_eye_inner_corner_x',
       'left_eye_inner_corner_y', 'left_eye_outer_corner_x',
       'left_eye_outer_corner_y', 'right_eye_inner_corner_x',
       'right_eye_inner_corner_y', 'right_eye_outer_corner_x',
       'right_eye_outer_corner_y', 'left_eyebrow_inner_end_x',
       'left_eyebrow_inner_end_y', 'left_eyebrow_outer_end_x',
       'left_eyebrow_outer_end_y', 'right_eyebrow_inner_end_x',
       'right_eyebrow_inner_end_y', 'right_eyebrow_outer_end_x',
       'right_eyebrow_outer_end_y', 'nose_tip_x', 'nose_tip_y',
       'mouth_left_corner_x', 'mouth_left_corner_y', 'mouth_right_corner_x',
       'mouth_right_corner_y', 'mouth_center_top_lip_x',
       'mouth_center_top_lip_y', 'mouth_center_bottom_lip_x',
       'mouth_center_bottom_lip_y', 'Image'],
      dtype='object')

In [ ]:
# Extract the image data and keypoint coordinates
images = train_data['Image'].apply(lambda x: np.array(x.split(' '), dtype=int))

In [ ]:
null_counts=train_data.isna().sum()
# null_counts

In [ ]:
non_much_Null_values=null_counts.index
non_much_Null_values = [col for col in non_much_Null_values if col != 'Image']
# non_much_Null_values

In [ ]:
# import matplotlib.pyplot as plt
# # Assuming your DataFrame is named 'df'
# for feature in non_much_Null_values:
#     train_data[feature].hist(bins=30)
#     plt.xlabel(feature)
#     plt.ylabel('Frequency')
#     plt.title('Distribution of {}'.format(feature))
#     plt.show()

In [ ]:
for feature in non_much_Null_values:
    median_value = train_data[feature].median()
    train_data[feature].fillna(median_value, inplace=True)

In [ ]:
keypoints = train_data.drop('Image', axis=1)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(images, keypoints, test_size=0.2, random_state=42)

In [ ]:
# Reshape the image data
X_train = np.array([x.reshape(96, 96, 1) for x in X_train])
X_val = np.array([x.reshape(96, 96, 1) for x in X_val])

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetV2S
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [ ]:
# Function to convert grayscale images to RGB
def gray_to_rgb(images):
    return np.stack([images] * 3, axis=-1)

In [ ]:
# Resize images to 260x260 and convert to RGB
images_resized = tf.image.resize(train_data['image'], [260, 260])
images_rgb = gray_to_rgb(images_resized)

In [ ]:
base_model = EfficientNetV2S(weights='imagenet', include_top=False, input_shape=(260, 260, 3))
base_model.trainable = False  # Freeze the base model

In [ ]:
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.1),
    layers.Dense(30)  # Assuming a binary classification problem
])

In [ ]:
# model = Sequential()

# # model.add(Convolution2D(32, (3,3), padding='same', use_bias=False, input_shape=(96,96,1)))
# # model.add(LeakyReLU(alpha = 0.1))
# # model.add(BatchNormalization())

# # model.add(Convolution2D(32, (3,3), padding='same', use_bias=False))
# # model.add(LeakyReLU(alpha = 0.1))
# # model.add(BatchNormalization())
# # model.add(MaxPool2D(pool_size=(2, 2)))

# # # model.add(Convolution2D(64, (3,3), padding='same', use_bias=False))
# # # model.add(LeakyReLU(alpha = 0.1))
# # # model.add(BatchNormalization())

# # # model.add(Convolution2D(64, (3,3), padding='same', use_bias=False))
# # # model.add(LeakyReLU(alpha = 0.1))
# # # model.add(BatchNormalization())
# # # model.add(MaxPool2D(pool_size=(2, 2)))

# # # model.add(Convolution2D(96, (3,3), padding='same', use_bias=False))
# # # model.add(LeakyReLU(alpha = 0.1))
# # # model.add(BatchNormalization())

# # # model.add(Convolution2D(96, (3,3), padding='same', use_bias=False))
# # # model.add(LeakyReLU(alpha = 0.1))
# # # model.add(BatchNormalization())
# # # model.add(MaxPool2D(pool_size=(2, 2)))

# # # model.add(Convolution2D(128, (3,3),padding='same', use_bias=False))
# # # # model.add(BatchNormalization())
# # # model.add(LeakyReLU(alpha = 0.1))
# # # model.add(BatchNormalization())

# # # model.add(Convolution2D(128, (3,3),padding='same', use_bias=False))
# # # model.add(LeakyReLU(alpha = 0.1))
# # # model.add(BatchNormalization())
# # # model.add(MaxPool2D(pool_size=(2, 2)))

# # # model.add(Convolution2D(256, (3,3),padding='same',use_bias=False))
# # # model.add(LeakyReLU(alpha = 0.1))
# # # model.add(BatchNormalization())

# # # model.add(Convolution2D(256, (3,3),padding='same',use_bias=False))
# # # model.add(LeakyReLU(alpha = 0.1))
# # # model.add(BatchNormalization())
# # # model.add(MaxPool2D(pool_size=(2, 2)))

# # # model.add(Convolution2D(512, (3,3), padding='same', use_bias=False))
# # # model.add(LeakyReLU(alpha = 0.1))
# # # model.add(BatchNormalization())

# # model.add(Convolution2D(512, (3,3), padding='same', use_bias=False))
# # model.add(LeakyReLU(alpha = 0.1))
# # model.add(BatchNormalization())


# model.add(Flatten())
# model.add(Dense(512,activation='relu'))
# model.add(Dropout(0.1))
# model.add(Dense(30))
# model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mae'])

In [ ]:
model.fit(X_train,y_train,epochs = 5,batch_size = 256,validation_split = 0.2)

Epoch 1/5
18/18 [==============================] - 231s 12s/step - loss: 520.2770 - mae: 16.2001 - val_loss: 2497.5452 - val_mae: 46.4705
Epoch 2/5
18/18 [==============================] - 211s 12s/step - loss: 93.2856 - mae: 7.5849 - val_loss: 2638.5972 - val_mae: 46.5951
Epoch 3/5
18/18 [==============================] - 217s 12s/step - loss: 57.7662 - mae: 5.9851 - val_loss: 2942.7329 - val_mae: 47.9032
Epoch 4/5
18/18 [==============================] - 210s 12s/step - loss: 45.5867 - mae: 5.3031 - val_loss: 3343.0193 - val_mae: 49.9338
Epoch 5/5
18/18 [==============================] - 217s 12s/step - loss: 41.1386 - mae: 5.0294 - val_loss: 3690.7939 - val_mae: 51.4732


In [ ]:
#preparing test data
timag = []
for i in range(0,1783):
    timg = test_data['Image'][i].split(' ')
    timg = ['0' if x == '' else x for x in timg]

    timag.append(timg)

In [ ]:
timage_list = np.array(timag,dtype = 'float')
X_test = timage_list.reshape(-1,96,96,1)

In [ ]:
pred = model.predict(X_test)

56/56 [==============================] - 19s 338ms/step


In [ ]:
lookid_data = pd.read_csv('IdLookupTable.csv')
lookid_list = list(lookid_data['FeatureName'])
imageID = list(lookid_data['ImageId']-1)
pre_list = list(pred)
rowid = lookid_data['RowId']
rowid=list(rowid)
feature = []
for f in list(lookid_data['FeatureName']):
    feature.append(lookid_list.index(f))
preded = []
for x,y in zip(imageID,feature):
    preded.append(pre_list[x][y])
rowid = pd.Series(rowid,name = 'RowId')
loc = pd.Series(preded,name = 'Location')
submission = pd.concat([rowid,loc],axis = 1)
submission.to_csv('face_key_detection_submission.csv',index = False)